<a href="https://colab.research.google.com/github/rahiakela/natural-language-processing-research-and-practice/blob/main/nlp-for-vector-similarity-search/02_nearest_neighbor_indexes_for_similarity_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Nearest Neighbor Indexes for Similarity Search

**Reference**

[Nearest Neighbor Indexes for Similarity Search](https://www.pinecone.io/learn/vector-indexes/)

##Setup

In [ ]:
!pip install faiss-cpu
!pip install -U sentence-transformers

In [4]:
import shutil
import urllib.request as request
from contextlib import closing
from io import StringIO
import pandas as pd
import numpy as np
import tarfile

from sentence_transformers import SentenceTransformer

import faiss

##Dataset

In [5]:
# first we download the Sift1M dataset
with closing(request.urlopen("ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz")) as res:
  with open("sift.tar.gz", "wb") as f:
    shutil.copyfileobj(res, f)

In [6]:
# download leaves us with a tar.gz file, we unzip it
tar = tarfile.open("sift.tar.gz", "r:gz")
tar.extractall()

In [7]:
# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
  a = np.fromfile(fp, "int32")
  d = a[0]
  return a.reshape(-1, d + 1)[:, 1:].copy().view("float32")

In [9]:
#  data we will search through
xb = read_fvecs("./sift/sift_base.fvecs")  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

In [10]:
xq.shape

(1, 128)

In [11]:
xb.shape

(1000000, 128)

In [12]:
xq

array([[  1.,   3.,  11., 110.,  62.,  22.,   4.,   0.,  43.,  21.,  22.,
         18.,   6.,  28.,  64.,   9.,  11.,   1.,   0.,   0.,   1.,  40.,
        101.,  21.,  20.,   2.,   4.,   2.,   2.,   9.,  18.,  35.,   1.,
          1.,   7.,  25., 108., 116.,  63.,   2.,   0.,   0.,  11.,  74.,
         40., 101., 116.,   3.,  33.,   1.,   1.,  11.,  14.,  18., 116.,
        116.,  68.,  12.,   5.,   4.,   2.,   2.,   9., 102.,  17.,   3.,
         10.,  18.,   8.,  15.,  67.,  63.,  15.,   0.,  14., 116.,  80.,
          0.,   2.,  22.,  96.,  37.,  28.,  88.,  43.,   1.,   4.,  18.,
        116.,  51.,   5.,  11.,  32.,  14.,   8.,  23.,  44.,  17.,  12.,
          9.,   0.,   0.,  19.,  37.,  85.,  18.,  16., 104.,  22.,   6.,
          2.,  26.,  12.,  58.,  67.,  82.,  25.,  12.,   2.,   2.,  25.,
         18.,   8.,   2.,  19.,  42.,  48.,  11.]], dtype=float32)

##Flat Index

In [13]:
# dimensionality of Sift1M data
d = 128
# number of nearest neighbors to return
k = 10

In [14]:
# initialize IndexFlatL2 index
index = faiss.IndexFlatIP(d)
index.add(xb)

In [15]:
%%time

# search
D, I = index.search(xq, k)
print(I)

[[932085 934876 561813 708177 706771 695756 435345 701258 872728 455537]]
CPU times: user 54.7 ms, sys: 0 ns, total: 54.7 ms
Wall time: 61.1 ms


In [ ]:
# [f'{i}: {sentences[i]}' for i in I[0]]

['3853: A group of football players is running in the field',
 '3412: A group of people playing football is running in the field',
 '10113: Two groups of people are playing football',
 '4266: A person playing football is running past an official carrying a football']

## Locality Sensitive Hashing

In [ ]:
# we have 4 vectors to return (k) - so we initialize a zero array to hold them
vecs = np.zeros((k, d))

# then iterate through each ID from I and add the reconstructed vector to our zero-array
for i, val in enumerate(I[0].tolist()):
  vecs[i, :] = index.reconstruct(val)
  
vecs.shape

(4, 768)

In [ ]:
vecs[0][:100]

array([ 0.01627023,  0.22325909, -0.15037383, -0.30747274, -0.27122426,
       -0.105932  , -0.06460915,  0.0473821 , -0.73349071, -0.37657702,
       -0.76762801,  0.16902868,  0.53107685,  0.51176631,  1.14415824,
       -0.08562893, -0.67240089, -0.96637088,  0.02545471, -0.2155983 ,
       -1.25656641, -0.82982188, -0.09824977, -0.21850872,  0.5061025 ,
        0.10527933,  0.50396907,  0.65242976, -1.39458668,  0.65847468,
       -0.21525355, -0.22487433,  0.81818348,  0.08464285, -0.76141697,
       -0.28928307, -0.0982579 , -0.73046142,  0.07855832, -0.84354609,
       -0.59242088,  0.7747137 , -1.20920563, -0.22757971, -1.30733621,
       -0.23081483, -1.31322527,  0.01629104, -0.97285467,  0.19308169,
        0.47424543,  1.1892091 , -1.96741259, -0.70061135, -0.29638749,
        0.6053372 ,  0.6240744 , -0.70340365, -0.86754185,  0.17673104,
       -0.19170581, -0.02951968,  0.22623521, -0.16695444, -0.80402541,
       -0.45918944,  0.69675523, -0.249282  , -1.01478708, -0.92

##Partitioning The Index

In [ ]:
# how many cells
n_list = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, n_list)

In [ ]:
# train index on data before adding any data to the index
index.is_trained

False

In [ ]:
index.train(sentence_embeddings)
# check if index is now trained
index.is_trained

True

In [ ]:
index.add(sentence_embeddings)
# number of embeddings indexed
index.ntotal

14504

In [ ]:
# Let’s search again using the same indexed sentence embeddings and the same query vector
%%time

# search
D, I = index.search(xq, k)
print(I)

[[ 3853  3412 10113  4266]]
CPU times: user 589 µs, sys: 16 µs, total: 605 µs
Wall time: 833 µs


In [ ]:
[f'{i}: {sentences[i]}' for i in I[0]]

['3853: A group of football players is running in the field',
 '3412: A group of people playing football is running in the field',
 '10113: Two groups of people are playing football',
 '4266: A person playing football is running past an official carrying a football']

In [ ]:
# We can increase the number of nearby cells to search too with nprobe.
index.nprobe = 10

In [ ]:
%%time

# search
D, I = index.search(xq, k)
print(I)

[[ 3853  3412 10113  4266]]
CPU times: user 1.08 ms, sys: 3 µs, total: 1.08 ms
Wall time: 1.09 ms


In [ ]:
[f'{i}: {sentences[i]}' for i in I[0]]

['3853: A group of football players is running in the field',
 '3412: A group of people playing football is running in the field',
 '10113: Two groups of people are playing football',
 '4266: A person playing football is running past an official carrying a football']

##Vector Reconstruction

In [ ]:
# now, to reconstruct the vectors, we must first create these direct mappings
index.make_direct_map()

In [ ]:
index.reconstruct(11122)[:100]

array([ 1.2838421 ,  0.21651636,  0.9712451 ,  0.31156752,  0.28217337,
        0.91760635,  1.5379874 ,  0.18022525, -0.87787133, -0.02692633,
       -1.3939755 , -0.02931822, -0.0910926 ,  0.8252405 ,  1.1967825 ,
        0.46976212, -0.4116232 , -0.44104576,  0.00642029,  0.14851105,
       -0.43181732,  0.53935724,  0.6806572 , -1.2024508 , -0.5175477 ,
       -0.87696105, -0.14817108, -0.24578056, -1.1521579 ,  0.2925653 ,
        0.1566306 ,  0.9666366 ,  0.48437908, -0.14077239,  0.6266316 ,
       -0.28244448, -0.12196147, -1.0980004 ,  0.39501837, -0.4538445 ,
        0.03117619,  0.350108  ,  0.088641  , -0.04181439,  0.49622074,
       -0.02361662,  2.3334348 , -0.49966168, -0.1949345 , -0.37520537,
        0.02379034, -0.2796866 ,  1.5051224 ,  0.75192493, -0.23107927,
       -0.4343509 ,  0.41142032, -0.58381236,  0.6106839 ,  0.5567988 ,
        0.39692488, -0.09096396,  0.01875396,  0.06579515, -0.54263747,
       -0.29689386, -0.2615427 , -0.36894163,  0.02824443, -0.37

##Quantization

In [ ]:
# number of centroid IDs in final compressed vectors
m = 8

# number of bits in each centroid
bits = 8

# we keep the same L2 distance flat index
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFPQ(quantizer, d, n_list, m, bits)

In [ ]:
index.is_trained

True

In [ ]:
index.train(sentence_embeddings)

In [ ]:
index.add(sentence_embeddings)

In [ ]:
# now we’re ready to begin searching using our new index
index.nprobe = 10

In [ ]:
%%time

D, I = index.search(xq, k)
print(I)

[[ 3853 18357  3412 17916]]
CPU times: user 3.63 ms, sys: 46 µs, total: 3.68 ms
Wall time: 2.66 ms


In [ ]:
[f'{i}: {sentences[i]}' for i in [3853, 3412]]

['3853: A group of football players is running in the field',
 '3412: A group of people playing football is running in the field']